# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [28]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [29]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"
#MODEL = "deepseek-r1:32b"

In [30]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]


In [31]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [7]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [32]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as text, images, videos, and music. This can be used to automate content creation for social media platforms, blogs, and websites.
2. **Marketing Automation**: Generative AI can help create personalized marketing campaigns by generating targeted advertisements, product descriptions, and customer communications.
3. **Product Design**: Generative AI can assist in designing new products, such as furniture, electronics, or fashion items, using computer-aided design (CAD) software.
4. **Customer Service Chatbots**: Generative AI-powered chatbots can provide personalized customer support, answer common queries, and route complex issues to human representatives.
5. **Data Analysis and Visualization**: Generative AI can generate insights and visualizations from large datasets, helping businesses make data-driven decisions.
6. **Cr

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [14]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: AI-powered tools can generate high-quality content such as articles, social media posts, product descriptions, and more, at scale and with minimal human intervention.
2. **Marketing Automation**: Generative AI can help automate marketing efforts by generating personalized email campaigns, ad copy, and social media content based on customer data and behavior.
3. **Product Design and Development**: AI-generated designs can be used to create new product concepts, optimize existing products, and reduce the time and cost associated with traditional design processes.
4. **Data Analysis and Visualization**: Generative AI can help analyze large datasets, identify patterns, and generate visualizations that provide insights into customer behavior, market trends, and other business metrics.
5. **Customer Service and Support**: AI-powered chatbots can use generative models to respond 

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [19]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

NameError: name 'ollama_via_openai' is not defined

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [20]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [21]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        
        # Store image URLs and their associated descriptions/stats
        self.items = []
        for img in soup.find_all('img'):
            if img.get('src'):
                img_url = img['src']
                # Convert relative URLs to absolute URLs
                if not img_url.startswith(('http://', 'https://')):
                    img_url = url + img_url
                
                # Find the parent element that contains both image and description
                parent = img.parent
                description = ""
                stats = ""
                
                # Look for description and stats in nearby elements
                if parent:
                    # Try to find description in a nearby paragraph or div
                    desc_elem = parent.find_next(['p', 'div'], class_=['description', 'item-description'])
                    if desc_elem:
                        description = desc_elem.get_text(strip=True)
                    
                    # Try to find stats in a nearby table or list
                    stats_elem = parent.find_next(['table', 'ul', 'ol'])
                    if stats_elem:
                        stats = stats_elem.get_text(strip=True)
                
                self.items.append({
                    'image_url': img_url,
                    'description': description,
                    'stats': stats
                })
        
        # Remove irrelevant elements
        for irrelevant in soup.body(["script", "style", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [22]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """You are an assistant that analyzes the contents of a videogame website, called Ragnarok Online 
and provides a table like output displaying the top 10 list items in the list, you will be able to identify the correct list because this items will have an image before its name and description,
If it includes a valid img url display the image in the output as well as the rest of the item information.
When displaying images, use markdown image syntax: ![alt text](image_url). And finally also grab the item iformation and display it as regular text"""

In [23]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please only get the top 10 list items in the list, you will be able to identify the correct list because this items will have an image before its name and description \
If it includes a valid img url display the image in the output as well as the rest of the item information.\n\n"
    user_prompt += website.text
    
    # Add image URLs to the prompt
    #if website.images:
        #user_prompt += "\n\nAvailable image URLs:\n"
        #for img_url in website.images:
            #user_prompt += f"- {img_url}\n"
            
    return user_prompt

In [24]:
ed = Website("https://irowiki.org/wiki/Mace")
#print(ed.title)
#print(ed.text)
#print(user_prompt_for(ed))

In [25]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [26]:
#messages_for(ed)

#def display_items(url):
#    website = Website(url)
#    messages = messages_for(website)
#    response = requests.post(OLLAMA_API, json={"model": MODEL, "messages": messages, "stream": False}, headers=HEADERS)
#    display(Markdown(response.json()['message']['content']))

def display_items(url):
    website = Website(url)
    messages = messages_for(website)
    response = requests.post(OLLAMA_API, json={"model": MODEL, "messages": messages, "stream": False}, headers=HEADERS)
    
    # Get the response content
    content = response.json()['message']['content']
    
    # Display the text content
    display(Markdown(content))
    
    # Display items with their images, descriptions, and stats
    if website.items:
        print("\nItems from the page:")
        for item in website.items:
            # Create a markdown table row with image and description
            item_markdown = f"| ![Image]({item['image_url']}) | {item['description']} |\n"
            if item['stats']:
                item_markdown += f"| Stats | {item['stats']} |\n"
            display(Markdown(item_markdown))

In [27]:
print(f"Model => {MODEL}")
display_items("https://irowiki.org/wiki/Mace")

Model => llama3.2


It appears that the provided text is a section from the iRO Wiki, a fan-made wiki for the MMORPG Elsword.

The section you provided seems to be a list of various job classes in the game, including their corresponding weapon types. The list includes:

1. Acolyte
2. Merchant
3. Priest

Each job class is listed with its corresponding weapon type, such as "Book" or "Mace".

Here's a summary of the job classes and weapons mentioned:

* Acolyte: Book
* Merchant: 
	+ No specific weapon type mentioned
* Priest: 
	+ Mace
	+ Rod

Please note that this section may not be exhaustive, and there may be additional job classes or weapon types listed elsewhere in the wiki.


Items from the page:


| ![Image](https://irowiki.org/images/db/item/16015.png) |  |
| Stats | Weaponsin Ragnarok OnlineWeaponryAxes•Books•Bows•Daggers•Guns•Huuma Shurikens•Instruments•Katars•Knuckles•Maces•One Handed Swords•Rods•Spears•Two Handed Swords•WhipsAmmunitionArrows•Bullets•Cannon Balls•Kunai•Shurikens |


| ![Image](https://irowiki.org/images/db/item/1501.png) |  |
| Stats | Weaponsin Ragnarok OnlineWeaponryAxes•Books•Bows•Daggers•Guns•Huuma Shurikens•Instruments•Katars•Knuckles•Maces•One Handed Swords•Rods•Spears•Two Handed Swords•WhipsAmmunitionArrows•Bullets•Cannon Balls•Kunai•Shurikens |


...

| ![Image](https://irowiki.org/images/db/item/1523.png) |  |
| Stats | Weaponsin Ragnarok OnlineWeaponryAxes•Books•Bows•Daggers•Guns•Huuma Shurikens•Instruments•Katars•Knuckles•Maces•One Handed Swords•Rods•Spears•Two Handed Swords•WhipsAmmunitionArrows•Bullets•Cannon Balls•Kunai•Shurikens |


| ![Image](https://irowiki.org/images/db/item/16021.png) |  |
| Stats | Weaponsin Ragnarok OnlineWeaponryAxes•Books•Bows•Daggers•Guns•Huuma Shurikens•Instruments•Katars•Knuckles•Maces•One Handed Swords•Rods•Spears•Two Handed Swords•WhipsAmmunitionArrows•Bullets•Cannon Balls•Kunai•Shurikens |


| ![Image](https://irowiki.org/images/db/item/16020.png) |  |
| Stats | Weaponsin Ragnarok OnlineWeaponryAxes•Books•Bows•Daggers•Guns•Huuma Shurikens•Instruments•Katars•Knuckles•Maces•One Handed Swords•Rods•Spears•Two Handed Swords•WhipsAmmunitionArrows•Bullets•Cannon Balls•Kunai•Shurikens |


| ![Image](https://irowiki.org/wiki/Mace/w/resources/assets/licenses/gnu-fdl.png) |  |


| ![Image](https://irowiki.org/wiki/Mace/w/resources/assets/poweredby_mediawiki_88x31.png) |  |
